In [1]:
# !pip install mlxtend==0.19.0

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

iris = load_iris()
x = iris.data
y = iris.target
knn = KNeighborsClassifier(n_neighbors=4)

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

## Forward Selection 

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs = SFS(knn,
          k_features=3,
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=0,
          n_jobs=-1
            )

feature_names = ('sepal length','sepal width','petal length','petal width')
sfs =sfs.fit(x,y,custom_feature_names=feature_names)

In [ ]:
sfs.get_metric_dict()

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

## Backward Selection 

In [ ]:
sfs = SFS(knn,
          k_features=2,
          forward=False,
          scoring='accuracy',
          floating=False, # it'll do bidirectional wayif its true in big dataset
          cv=2,
          n_jobs=1
         )
sfs =sfs.fit(x,y,custom_feature_names=feature_names)

In [ ]:
pd.DataFrame().from_dict(sfs.get_metric_dict()).T

# Recursive Feature Elimination

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
dataset=load_boston()
df=pd.DataFrame(dataset.data)
df.columns=dataset.feature_names
df.target=np.asarray(dataset.target)
df['price']=pd.Series(df.target)
df.head()

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [ ]:
lin_reg = LinearRegression()

In [ ]:
rfe = RFE(lin_reg)
myvalues = rfe.fit(X_train,y_train)
myvalues.support_
myvalues.ranking_

print("no. of features :  %s" %(myvalues.n_features_))
print("Selected features :  %s" %(myvalues.support_))
print("feature Ranking :  %s" %(myvalues.ranking_))